In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import LSTM,Dense, Concatenate,Bidirectional,Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import AdditiveAttention
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow import Tensor
import tensorflow.keras.backend as K

In [2]:
#importing vocabulary
ds_path='/home/al/Desktop/nnfl/word2vecmod.txt'
data = np.loadtxt(ds_path,dtype=str,usecols=(0))
print(data)
print(data.shape[0])

['a' 'abandon' 'ability' ... '<start>' '<end>' '<pad>']
5035


In [3]:
#loading word vectors for getting unkown words
tmp_file = get_tmpfile(datapath("/home/al/Desktop/nnfl/new"))
word_model = KeyedVectors.load(tmp_file)

In [4]:
#list of words in word vectors
data2= np.loadtxt('/home/al/Desktop/nnflproj/glove.6B.300d.txt',dtype=str,usecols=(0))

In [5]:
#making word-to-index and index-to-word dictionaries
word_to_ix=dict(zip(data,range(0,data.shape[0])))
ix_to_word=dict(zip(range(0,data.shape[0]),data))

In [6]:
#constants
voc=data.shape[0]#for <start>,<end>,<pad>
ip_len=10
op_len=10
n_s=64+op_len#no of decoder lstm
n_a=64#no 


In [7]:
#one-hot function
def oh(Xq,ip_ln):
    Xoh=np.zeros([Xq.shape[0],ip_ln,voc])
    for i in range(Xq.shape[0]):
        for j in range(ip_ln):
            Xoh[i,j,int(Xq[i,j])]=1
    return Xoh

In [8]:
#model
enc_ip=Input(shape=(ip_len,voc))
enc_lstm=LSTM(256,return_state=True,return_sequences=True)
enc_op,h,c=enc_lstm(enc_ip)
concat=Concatenate(axis=-1)
enc_states=[h,c]
dec_ip=Input(shape=(op_len,voc))
dec_lstm=LSTM(256,return_sequences=True,return_state=True)
dec_op,_,_=dec_lstm(dec_ip,initial_state=enc_states)
att=AdditiveAttention()([dec_op, enc_op])
dense=Dense(voc,activation='softmax')
op=concat([dec_op,att])
op=dense(op)
model=Model([enc_ip,dec_ip],op)
model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 5035)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 10, 5035)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 10, 256), (N 5419008     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 10, 256), (N 5419008     input_2[0][0]                    
                                                                 lstm[0][1]                   

In [43]:
#creating checkpoint
filepath="/home/al/Desktop/nnfl final/wfin.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_accuracy',verbose=1,save_best_only=True)
#model.load_weights(filepath)

In [55]:
d_path='/home/al/Desktop/nnflproj/nds/nds-a'
idx=0
idx2=0
for e  in 'b':
#bcdefghi':

    for f  in 'abc':
        ds = np.loadtxt(d_path+e+f+'.csv',delimiter='\n',dtype='str')
        num_sen=1000
        idx=0
        Xtr=np.ndarray(shape=[num_sen,ip_len])
        for j in range(0,num_sen):
                #ds.shape[0]-(ds.shape[0]%1000)+1)
                w1=np.array(ds[j].lower().split())
                for i in range(0,ip_len):    
                    w=w1[i]
                    if i==ip_len-1:
                        #w1_sum/=50
                        Xtr[j,i]=voc-2#for<end>
                        
                        break
                    if w=='<start>':
                        Xtr[j,i]=voc-3
                        continue
                    if w=='<end>':
                        Xtr[j,i]=voc-2#<end>
                        i+=1
                        while i<ip_len:
                            Xtr[j,i]=voc-1#padding
                            i+=1
                        break
                    if w in data:                
                        Xtr[j,i]=word_to_ix[w]
                    else:
                        try:
                            temp=word_model.most_similar(positive=[w],negative=[],restrict_vocab=4095)#for<unk>
                            idx2+=1
                            Xtr[j,i]=word_to_ix[temp[0][0]]
                        except KeyError:
                            Xtr[j,i]=word_to_ix['it']
                            idx+=1


                        
                           
                if j%1000==0:
                    print(j)

#train
        for kl in range(50,num_sen,50):
            u=oh(Xtr[kl-50:kl+1],ip_len)
            u=tf.convert_to_tensor(u)
            model.fit([u[0:-1],u[0:-1]],u[1:,:,:],epochs=70,batch_size=20,use_multiprocessing=True,callbacks=[checkpoint],verbose=2,validation_split=0.2,shuffle=True)
print('\n\n\n\n\ndone')

5/70

Epoch 00035: val_accuracy did not improve from 0.84000
40/40 - 1s - loss: 0.1398 - accuracy: 0.9925 - val_loss: 6.6533 - val_accuracy: 0.1900
Epoch 36/70

Epoch 00036: val_accuracy did not improve from 0.84000
40/40 - 1s - loss: 0.1389 - accuracy: 0.9925 - val_loss: 6.6542 - val_accuracy: 0.2000
Epoch 37/70

Epoch 00037: val_accuracy did not improve from 0.84000
40/40 - 1s - loss: 0.1381 - accuracy: 0.9925 - val_loss: 6.6555 - val_accuracy: 0.2000
Epoch 38/70

Epoch 00038: val_accuracy did not improve from 0.84000
40/40 - 1s - loss: 0.1374 - accuracy: 0.9925 - val_loss: 6.6570 - val_accuracy: 0.2000
Epoch 39/70

Epoch 00039: val_accuracy did not improve from 0.84000
40/40 - 1s - loss: 0.1368 - accuracy: 0.9925 - val_loss: 6.6592 - val_accuracy: 0.2000
Epoch 40/70

Epoch 00040: val_accuracy did not improve from 0.84000
40/40 - 1s - loss: 0.1362 - accuracy: 0.9925 - val_loss: 6.6611 - val_accuracy: 0.2100
Epoch 41/70

Epoch 00041: val_accuracy did not improve from 0.84000
40/40 - 1

KeyboardInterrupt: 

In [57]:
def ev(s1):
        s1='<start> '+s1+' <end>'
        Xt=np.zeros(shape=[ip_len])
        #ds.shape[0]-(ds.shape[0]%1000)+1)
        w1=np.array(s1.lower().split())
        for i in range(0,ip_len):    
            w=w1[i]
            if i==ip_len-1:
                #w1_sum/=50
                Xt[i]=voc-2#for<end>
                
                break
            if w=='<start>':
                Xt[i]=voc-3
                continue
            if w=='<end>':
                Xt[i]=voc-2#<end>
                i+=1
                while i<ip_len:
                    Xt[i]=voc-1#padding
                    i+=1
                break
            if w in data:                
                Xt[i]=word_to_ix[w]
            else:
                try:
                    temp=word_model.most_similar(positive=[w],negative=[],restrict_vocab=4095)#for<unk>
                    idx2+=1
                    Xt[i]=word_to_ix[temp[0][0]]
                except KeyError:

                    Xt[i]=word_to_ix['it']
                    idx+=1
            return Xt.reshape([1,ip_len])


In [63]:
Xt=ev('man')

#test
ut=oh(Xt[0:1],ip_len)
a=model.predict([ut,ut])
b=np.argmax(a,axis=-1)
print(b)
for el in b.flatten():
        print(ix_to_word[el])

[[5032 2686 5023 2934 2709 2709 1799 1799 5033 5033]]
<start>
the
,
what
this
this
of
of
<end>
<end>
